<span style="font-size:2em; font-weight:bold;">ESN-ONLY</span>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose

# === Publication Style ===
sns.set(style="whitegrid", font_scale=1.2)

# === Save Directory ===
save_dir = r"C:\Users\Adarsh Pradeep\OneDrive\Desktop\Adarsh_Personal\PAPERS_TO_PUBLISH\plotsresults"
os.makedirs(save_dir, exist_ok=True)

# === Load and Preprocess Data ===
df = pd.read_csv("wind_paper.csv")
df = df[df['Patv'] > 0].drop(['Tmstamp', 'TurbID'], axis=1)
N_LAGS = 36
for lag in range(1, N_LAGS + 1):
    df[f'Patv_lag_{lag}'] = df['Patv'].shift(lag)
df['Patv_diff'] = df['Patv'].diff()
df['RollingMean'] = df['Patv'].rolling(window=6).mean()
df.dropna(inplace=True)

X = df.drop('Patv', axis=1).values
y = df['Patv'].values.reshape(-1, 1)
X = MinMaxScaler().fit_transform(X)
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y_scaled[:split], y_scaled[split:]

# === ESN Class ===
class ESN:
    def __init__(self, input_dim, reservoir_size=2000, spectral_radius=0.98, sparsity=0.05,
                 reg=1e-3, leaky_rate=0.15, washout=150):
        self.input_dim = input_dim + 1
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.reg = reg
        self.leaky_rate = leaky_rate
        self.washout = washout
        self.Win = np.random.uniform(-0.5, 0.5, (reservoir_size, self.input_dim))
        W = np.random.rand(reservoir_size, reservoir_size) - 0.5
        W[np.random.rand(*W.shape) > sparsity] = 0
        eig_val = max(abs(np.linalg.eigvals(W)))
        self.W = W * (spectral_radius / eig_val)
    def _update(self, state, input_):
        u = np.concatenate((input_, [1]))
        pre_activation = np.dot(self.Win, u) + np.dot(self.W, state)
        return (1 - self.leaky_rate) * state + self.leaky_rate * np.tanh(pre_activation)
    def fit(self, X, y):
        states = []
        state = np.zeros(self.reservoir_size)
        for x in X:
            state = self._update(state, x)
            states.append(state.copy())
        states = np.array(states)
        X_eff, y_eff = states[self.washout:], y[self.washout:]
        self.Wout = Ridge(alpha=self.reg, fit_intercept=False).fit(X_eff, y_eff).coef_.T.flatten()
    def predict(self, X):
        preds, state = [], np.zeros(self.reservoir_size)
        for x in X:
            state = self._update(state, x)
            preds.append(state @ self.Wout)
        return np.array(preds)

# === Train & Predict ===
start = time.time()
esn = ESN(input_dim=X.shape[1])
esn.fit(X_train, y_train)
y_pred_scaled = esn.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_actual = y_scaler.inverse_transform(y_test)
end = time.time()

# === Metrics ===
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_actual, y_pred)
mape = np.mean(np.abs((y_test_actual - y_pred) / y_test_actual)) * 100
r2 = r2_score(y_test_actual, y_pred)
train_time = end - start

print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, R2: {r2:.4f}, TrainTime: {train_time:.2f}s")

df_test = df.iloc[split:].copy()
df_test['Error'] = abs(df_test['Patv'].values - y_pred.flatten())

# === Plots Section (Auto-save, 300 DPI, .jpg) ===

# 1. Forecast vs Actual
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:100], label='Actual', color='black')
plt.plot(y_pred[:100], label='Predicted', color='orange')
plt.title("Forecast vs Actual (First 100 Samples)", fontsize=22)
plt.xlabel("Sample Index", fontsize=20)
plt.ylabel("Power Output (Patv) [kW]", fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_forecast_vs_actual.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 2. Residual Distribution
plt.figure(figsize=(8, 5))
sns.histplot(df_test['Error'], kde=True, color='red')
plt.title("Residual Distribution", fontsize=22)
plt.xlabel("Absolute Error (kW)", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_residual_distribution.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 3. Error Over Time
plt.figure(figsize=(12, 4))
plt.plot(df_test['Error'][:300], label='Error', color='purple')
plt.title("Prediction Error Over Time", fontsize=22)
plt.xlabel("Sample Index", fontsize=20)
plt.ylabel("Absolute Error (kW)", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_error_over_time.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 4. Scatter Plot: Actual vs Predicted
plt.figure(figsize=(6, 6))
plt.scatter(y_test_actual, y_pred, alpha=0.4)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--')
plt.title("Scatter Plot: Actual vs Predicted", fontsize=22)
plt.xlabel("Actual Power Output (kW)", fontsize=20)
plt.ylabel("Predicted Power Output (kW)", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_scatter_actual_pred.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 5. Lag Correlation Plot (Autocorrelation)
plt.figure(figsize=(8, 4))
ax = plt.gca()
plot_acf(df['Patv'], lags=30, ax=ax)
plt.title("Lag Correlation Plot (Autocorrelation)", fontsize=22)
plt.xlabel("Lag", fontsize=20)
plt.ylabel("Autocorrelation", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_autocorrelation_patv.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 6. Error Box Plot by Month
df_test['Month'] = pd.to_datetime(df_test.index, errors='coerce').month
month_order = [1,2,3,4,5,6,7,8,9,10,11,12]
month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.figure(figsize=(14, 7))
sns.boxplot(x='Month', y='Error', data=df_test, hue='Month', palette="Set2",
            order=month_order, dodge=False, legend=False)
plt.xticks(ticks=range(12), labels=month_names, fontsize=18, rotation=45)
plt.title("Error Box Plot by Month", fontsize=22)
plt.xlabel("Month", fontsize=20)
plt.ylabel("Absolute Error (kW)", fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_error_boxplot_by_month.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 7. Box Plot of Errors (per 500 samples)
chunks = len(df_test) // 500
residuals_trimmed = df_test['Error'].values[:chunks * 500]
plt.figure(figsize=(12, 7))
sns.boxplot(data=pd.DataFrame(residuals_trimmed.reshape(-1, 500)).T, color='skyblue')
plt.title("Box Plot of Errors (per 500 samples)", fontsize=22)
plt.xlabel("Chunk Index", fontsize=20)
plt.ylabel("Absolute Error (kW)", fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_boxplot_errors_per500.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 8. Feature Importance (Ridge Coef Magnitude)
feature_importance = np.abs(esn.Wout)
plt.figure(figsize=(10, 5))
plt.bar(range(len(feature_importance)), feature_importance, color='seagreen')
plt.title("Feature Importance (Ridge Coef Magnitude)", fontsize=22)
plt.xlabel("Feature Index", fontsize=20)
plt.ylabel("Importance", fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "esn_feature_importance.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 9. Time-Series Decomposition
decomp = seasonal_decompose(df['Patv'].values[:500], model='additive', period=24)
fig = decomp.plot()
fig.set_size_inches(12, 8)
plt.suptitle("Time-Series Decomposition of Power Output (Patv)", fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(save_dir, "esn_ts_decomposition_patv.jpg"), dpi=300, bbox_inches='tight')
plt.close()


KeyboardInterrupt: 

<span style="font-size:2em; font-weight:bold;">gradient_boost</span>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose

# Set style for publication
sns.set(style="whitegrid", font_scale=1.2)

# 1. Directory Setup
save_dir = r"C:\Users\Adarsh Pradeep\OneDrive\Desktop\Adarsh_Personal\PAPERS_TO_PUBLISH\plotsresults"
os.makedirs(save_dir, exist_ok=True)

# 2. Load and Preprocess Data
df = pd.read_csv("wind_paper.csv")
df = df[df['Patv'] > 0].drop(['Tmstamp', 'TurbID'], axis=1)
N_LAGS = 36
for lag in range(1, N_LAGS + 1):
    df[f'Patv_lag_{lag}'] = df['Patv'].shift(lag)
df['Patv_diff'] = df['Patv'].diff()
df['RollingMean'] = df['Patv'].rolling(window=6).mean()
df.dropna(inplace=True)

X = df.drop('Patv', axis=1).values
y = df['Patv'].values.reshape(-1, 1)
X = MinMaxScaler().fit_transform(X)
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y_scaled[:split], y_scaled[split:]

# 3. ESN Class Definition
class ESN:
    def __init__(self, input_dim, reservoir_size=2000, spectral_radius=0.98, sparsity=0.05,
                 reg=1e-3, leaky_rate=0.15, washout=150):
        self.input_dim = input_dim + 1
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.reg = reg
        self.leaky_rate = leaky_rate
        self.washout = washout
        self.Win = np.random.uniform(-0.5, 0.5, (reservoir_size, self.input_dim))
        W = np.random.rand(reservoir_size, reservoir_size) - 0.5
        W[np.random.rand(*W.shape) > sparsity] = 0
        eig_val = max(abs(np.linalg.eigvals(W)))
        self.W = W * (spectral_radius / eig_val)

    def _update(self, state, input_):
        u = np.concatenate((input_, [1]))
        pre_activation = np.dot(self.Win, u) + np.dot(self.W, state)
        return (1 - self.leaky_rate) * state + self.leaky_rate * np.tanh(pre_activation)

    def fit(self, X, y):
        states = []
        state = np.zeros(self.reservoir_size)
        for x in X:
            state = self._update(state, x)
            states.append(state.copy())
        states = np.array(states)
        X_eff, y_eff = states[self.washout:], y[self.washout:]
        self.Wout = Ridge(alpha=self.reg, fit_intercept=False).fit(X_eff, y_eff).coef_.T.flatten()

    def predict(self, X):
        preds, state = [], np.zeros(self.reservoir_size)
        for x in X:
            state = self._update(state, x)
            preds.append(state @ self.Wout)
        return np.array(preds)

# 4. Training & Prediction
start = time.time()
esn = ESN(input_dim=X.shape[1])
esn.fit(X_train, y_train)
y_pred_scaled = esn.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_actual = y_scaler.inverse_transform(y_test)
end = time.time()

# 5. Metrics Calculation
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_actual, y_pred)
mape = np.mean(np.abs((y_test_actual - y_pred) / y_test_actual)) * 100
r2 = r2_score(y_test_actual, y_pred)
train_time = end - start
print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, R2: {r2:.4f}, TrainTime: {train_time:.2f}s")

df_test = df.iloc[split:].copy()
df_test['Error'] = abs(df_test['Patv'].values - y_pred.flatten())

# === PLOTS SECTION: All plots save in .jpg, dpi=300 ===

# 1. Forecast vs Actual
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:100], label='Actual', color='black')
plt.plot(y_pred[:100], label='Predicted', color='orange')
plt.title("Forecast vs Actual (First 100 Samples)", fontsize=22)
plt.xlabel("Sample Index", fontsize=20)
plt.ylabel("Power Output (Patv) [kW]", fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "forecast_vs_actual.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 2. Residual Distribution
plt.figure(figsize=(8, 5))
sns.histplot(df_test['Error'], kde=True, color='red')
plt.title("Residual Distribution", fontsize=22)
plt.xlabel("Absolute Error (kW)", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "residual_distribution.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 3. Error Over Time
plt.figure(figsize=(12, 4))
plt.plot(df_test['Error'][:300], label='Error', color='purple')
plt.title("Prediction Error Over Time", fontsize=22)
plt.xlabel("Sample Index", fontsize=20)
plt.ylabel("Absolute Error (kW)", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "error_over_time.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 4. Scatter Plot: Actual vs Predicted
plt.figure(figsize=(6, 6))
plt.scatter(y_test_actual, y_pred, alpha=0.4)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--')
plt.title("Scatter Plot: Actual vs Predicted", fontsize=22)
plt.xlabel("Actual Power Output (kW)", fontsize=20)
plt.ylabel("Predicted Power Output (kW)", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "scatter_actual_vs_predicted.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 5. Lag Correlation Plot (Autocorrelation)
plt.figure(figsize=(8, 4))
ax = plt.gca()
plot_acf(df['Patv'], lags=30, ax=ax)
plt.title("Lag Correlation Plot (Autocorrelation of Power Output)", fontsize=22)
plt.xlabel("Lag", fontsize=20)
plt.ylabel("Autocorrelation", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "autocorrelation_patv.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 6. Error Box Plot by Month
df_test['Month'] = pd.to_datetime(df_test.index, errors='coerce').month
month_order = [1,2,3,4,5,6,7,8,9,10,11,12]
month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.figure(figsize=(14, 7))
sns.boxplot(x='Month', y='Error', data=df_test, hue='Month', palette="Set2",
            order=month_order, dodge=False, legend=False)
plt.xticks(ticks=range(12), labels=month_names, fontsize=18, rotation=45)
plt.title("Error Box Plot by Month", fontsize=22)
plt.xlabel("Month", fontsize=20)
plt.ylabel("Absolute Error (kW)", fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "error_boxplot_by_month.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 7. Box Plot of Errors (per 500 samples)
chunks = len(df_test) // 500
residuals_trimmed = df_test['Error'].values[:chunks * 500]
plt.figure(figsize=(12, 7))
sns.boxplot(data=pd.DataFrame(residuals_trimmed.reshape(-1, 500)).T, color='skyblue')
plt.title("Box Plot of Errors (per 500 samples)", fontsize=22)
plt.xlabel("Chunk Index", fontsize=20)
plt.ylabel("Absolute Error (kW)", fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "boxplot_errors_per500.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 8. Feature Importance (Ridge Coef Magnitude)
feature_importance = np.abs(esn.Wout)
plt.figure(figsize=(10, 5))
plt.bar(range(len(feature_importance)), feature_importance, color='seagreen')
plt.title("Feature Importance (Ridge Coef Magnitude)", fontsize=22)
plt.xlabel("Feature Index", fontsize=20)
plt.ylabel("Importance", fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "feature_importance.jpg"), dpi=300, bbox_inches='tight')
plt.close()

# 9. Time-Series Decomposition
decomp = seasonal_decompose(df['Patv'].values[:500], model='additive', period=24)
fig = decomp.plot()
fig.set_size_inches(12, 8)
plt.suptitle("Time-Series Decomposition of Power Output (Patv)", fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(save_dir, "ts_decomposition_patv.jpg"), dpi=300, bbox_inches='tight')
plt.close()


<span style="font-size:2em; font-weight:bold;">QUANTUM_AI</span>

In [ ]:
import pennylane as qml
from pennylane.qnn import KerasLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Set reproducibility for results
np.random.seed(42)
tf.random.set_seed(42)

# Set number of qubits
nq = 4
dev = qml.device("default.qubit", wires=nq)

# Make QNode TF-aware
@qml.qnode(dev, interface="tf")
def qc(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(nq), rotation='Y')
    qml.templates.StronglyEntanglingLayers(weights, wires=range(nq))
    return [qml.expval(qml.PauliZ(i)) for i in range(nq)]

# Define weight shapes for the PQC
weight_shapes = {"weights": (3, nq, 3)}

# Create the quantum Keras layer
qlayer = KerasLayer(qc, weight_shapes=weight_shapes, output_dim=nq)

# Build the full hybrid model
model = Sequential([
    Flatten(input_shape=(nq,)),
    Dense(nq, activation="relu"),
    qlayer,
    Dense(1, activation="linear"),
])

# Compile
model.compile(optimizer='adam', loss='mse')

# Generate synthetic training data (replace with real data as needed)
X = np.random.rand(100, nq)
y = np.random.rand(100, 1)

# Train and capture training history
history = model.fit(X, y, epochs=20, batch_size=10, verbose=1)

# Save training loss curve in high-resolution for publication
save_dir = r"C:\Users\Adarsh Pradeep\OneDrive\Desktop\Adarsh_Personal\PAPERS_TO_PUBLISH\plotsresults"
os.makedirs(save_dir, exist_ok=True)

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], marker='o', linewidth=3)
plt.title("QNN Training Loss Curve", fontsize=22)
plt.xlabel("Epoch", fontsize=20)
plt.ylabel("Loss (MSE)", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "qnn_training_loss_curve.jpg"), dpi=300, bbox_inches='tight')
plt.close()


ESN+DNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

sns.set(style="whitegrid", font_scale=1.2)
save_dir = r"C:\Users\Adarsh Pradeep\OneDrive\Desktop\Adarsh_Personal\PAPERS_TO_PUBLISH\plotsresults"
os.makedirs(save_dir, exist_ok=True)

# Load & Preprocess Data
df = pd.read_csv("wind_paper.csv")
df = df[df['Patv'] > 0].drop(['Tmstamp', 'TurbID'], axis=1)
N_LAGS = 36
for lag in range(1, N_LAGS + 1):
    df[f'Patv_lag_{lag}'] = df['Patv'].shift(lag)
df['Patv_diff'] = df['Patv'].diff()
df['RollingMean'] = df['Patv'].rolling(window=6).mean()
df.dropna(inplace=True)

X = df.drop('Patv', axis=1).values
y = df['Patv'].values.reshape(-1, 1)
X = MinMaxScaler().fit_transform(X)
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y_scaled[:split], y_scaled[split:]

# ESN Class
class ESN:
    def __init__(self, input_dim, reservoir_size=900, spectral_radius=1.0, sparsity=0.05, reg=1e-5, leaky_rate=0.3, washout=60):
        self.input_dim = input_dim + 1
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.reg = reg
        self.leaky_rate = leaky_rate
        self.washout = washout
        self.Win = np.random.uniform(-0.5, 0.5, (reservoir_size, self.input_dim))
        W = np.random.rand(reservoir_size, reservoir_size) - 0.5
        W[np.random.rand(*W.shape) > sparsity] = 0
        eig_val = max(abs(np.linalg.eigvals(W)))
        self.W = W * (spectral_radius / eig_val)
    def _update(self, state, input_):
        u = np.concatenate((input_, [1]))
        pre_activation = np.dot(self.Win, u) + np.dot(self.W, state)
        return (1 - self.leaky_rate) * state + self.leaky_rate * np.tanh(pre_activation)
    def fit(self, X, y):
        states = []
        state = np.zeros(self.reservoir_size)
        for x in X:
            state = self._update(state, x)
            states.append(state.copy())
        states = np.array(states)
        X_eff, y_eff = states[self.washout:], y[self.washout:]
        self.Wout = Ridge(alpha=self.reg, fit_intercept=False).fit(X_eff, y_eff).coef_.T.flatten()
        self.trained_states = states
    def predict(self, X):
        preds, state = [], np.zeros(self.reservoir_size)
        for x in X:
            state = self._update(state, x)
            preds.append(state @ self.Wout)
        return np.array(preds)

# Hybrid/Dense model
def build_hybrid(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train ESN
start = time.time()
esn = ESN(input_dim=X.shape[1])
esn.fit(X_train, y_train)
esn_pred_train = esn.predict(X_train)
esn_pred_test = esn.predict(X_test)

# Hybrid Training
X_hybrid_train = esn_pred_train.reshape(-1, 1)
X_hybrid_test = esn_pred_test.reshape(-1, 1)
hybrid_model = build_hybrid(1)
hybrid_model.fit(X_hybrid_train, y_train, epochs=50, batch_size=32, verbose=0)
y_pred_scaled = hybrid_model.predict(X_hybrid_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_test_actual = y_scaler.inverse_transform(y_test)
end = time.time()

# Metrics
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_actual, y_pred)
mape = np.mean(np.abs((y_test_actual - y_pred) / y_test_actual)) * 100
r2 = r2_score(y_test_actual, y_pred)
train_time = end - start
print("Hybrid Model Metrics:")
print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, R2: {r2:.4f}, Train Time: {train_time:.2f}s")

# Plots
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:100], label='Actual', color='black')
plt.plot(y_pred[:100], label='Predicted', color='orange')
plt.title("Forecast vs Actual (Hybrid Model)", fontsize=22)
plt.xlabel("Sample Index", fontsize=20)
plt.ylabel("Power Output (Patv) [kW]", fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "hybrid_forecast_vs_actual.jpg"), dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(6, 6))
plt.scatter(y_test_actual, y_pred, alpha=0.4)
plt.plot([y_test_actual.min(), y_test_actual.max()], [y_test_actual.min(), y_test_actual.max()], 'k--')
plt.title("Hybrid Model: Actual vs Predicted", fontsize=22)
plt.xlabel("Actual Power Output (kW)", fontsize=20)
plt.ylabel("Predicted Power Output (kW)", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "hybrid_scatter_actual_pred.jpg"), dpi=300, bbox_inches='tight')
plt.close()


414/414 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step
Hybrid Model Metrics:
RMSE: 26.2048, MAE: 13.2853, MAPE: 10.67%, R2: 0.9953, Train Time: 140.05s
